In [1]:
# Imports

import pandas as pd
import json
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Read the two csv datasets

world_co2 = pd.read_csv("owid-co2-data.csv")
sector_co2 = pd.read_csv("ghg-emissions-by-sector.csv")

In [3]:
# Read the geojson file with countries and coordinates

with open("WorldMapGeo.js") as file:
     geo = json.loads(file.read())

In [4]:
# Common elements between the two datasets

names_one = list(world_co2['country'].value_counts().index)
names_two = list(sector_co2['Entity'].value_counts().index)
common_elements = set(names_one) & set(names_two)

# Keep the countries only
continents_reference = ['Africa', 'Antarctica', 'Asia', 'Oceania', 'Europe', 'North America', 'South America']
incomes_reference = ['Low-income countries', 'Lower-middle-income countries', 'Upper-middle-income countries', 'High-income countries']
continents = list(set(continents_reference) & set(common_elements))
incomes_classification = list(set(incomes_reference) & set(common_elements))
countries = list(((set(common_elements) ^ set(continents)) ^ set(incomes_classification)) ^ set(['World', 'European Union (27)']))
world_co2_overlap = world_co2[world_co2['country'].isin(common_elements)]
countries_co2 = world_co2_overlap[world_co2_overlap['country'].isin(countries)]
countries_sector = sector_co2[sector_co2['Entity'].isin(countries)]

# Keep only the time period 1990-2019
countries_co2 = countries_co2[countries_co2['year'].isin(np.linspace(1990, 2019, 30))]

# Set the same column names for the common columns and merge the two datasets
countries_sector.rename(columns={"Entity": "country", "Code": "iso_code", "Year": "year"}, inplace=True)
countries_co2.reset_index(drop=True, inplace=True)
countries_sector.reset_index(drop=True, inplace=True)
merged = pd.merge(countries_co2, countries_sector, how="inner", on=["country", "iso_code", "year"])
display(merged)

,country,year,iso_code,population,gdp,cement_co2,cement_co2_per_capita,co2,co2_growth_abs,co2_growth_prct,...,Land-use change and forestry,Waste,Industry,Manufacturing and construction,Transport,Electricity and heat,Buildings,Fugitive emissions,Other fuel combustion,Aviation and shipping
0,Afghanistan,1990,AFG,10694804.0,1.306598e+10,0.046,0.004,2.024,-0.741,-26.784,...,-2.390000e+06,1.230000e+06,5.000000e+04,4.100000e+05,1.670000e+06,1.700000e+05,8.000000e+04,280000.001192,0.000000,19999.999553
1,Afghanistan,1991,AFG,10745168.0,1.204736e+10,0.046,0.004,1.914,-0.110,-5.435,...,-2.390000e+06,1.320000e+06,6.000000e+04,4.000000e+05,1.550000e+06,1.700000e+05,7.000000e+04,239999.994636,0.000000,19999.999553
2,Afghanistan,1992,AFG,12057436.0,1.267754e+10,0.046,0.004,1.482,-0.432,-22.580,...,-2.390000e+06,1.400000e+06,6.000000e+04,3.700000e+05,7.700000e+05,1.600000e+05,3.000000e+04,200000.002980,0.000000,19999.999553
3,Afghanistan,1993,AFG,14003764.0,9.834581e+09,0.047,0.003,1.487,0.005,0.330,...,-2.390000e+06,1.490000e+06,6.000000e+04,3.600000e+05,7.400000e+05,1.600000e+05,3.000000e+04,159999.996424,0.000000,19999.999553
4,Afghanistan,1994,AFG,15455560.0,7.919857e+09,0.047,0.003,1.454,-0.033,-2.227,...,-2.390000e+06,1.580000e+06,6.000000e+04,3.400000e+05,7.100000e+05,1.600000e+05,2.000000e+04,119999.997318,0.000000,19999.999553
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5785,Zimbabwe,2015,ZWE,14154937.0,2.102745e+10,0.585,0.041,12.255,0.309,2.585,...,1.161000e+07,2.430000e+06,1.660000e+06,1.110000e+06,2.650000e+06,7.250000e+06,2.590000e+06,660000.026226,920000.016689,70000.000298
5786,Zimbabwe,2016,ZWE,14452705.0,2.096179e+10,0.461,0.032,10.533,-1.722,-14.047,...,8.740000e+07,2.480000e+06,1.780000e+06,1.110000e+06,2.250000e+06,6.260000e+06,2.650000e+06,680000.007153,800000.011921,129999.995232
5787,Zimbabwe,2017,ZWE,14751101.0,2.194784e+10,0.469,0.032,9.596,-0.937,-8.899,...,8.729000e+07,2.540000e+06,1.890000e+06,1.140000e+06,2.310000e+06,5.440000e+06,2.680000e+06,699999.988079,810000.002384,159999.996424
5788,Zimbabwe,2018,ZWE,15052191.0,2.271535e+10,0.558,0.037,11.795,2.199,22.920,...,8.738000e+07,2.590000e+06,1.970000e+06,1.200000e+06,2.970000e+06,6.630000e+06,2.740000e+06,709999.978542,939999.997616,189999.997616


In [5]:
# Replace country names in merged dataset to map with the geojson file
replace_countries = [
    ('Bahamas', 'The Bahamas'), 
    ('Congo', 'Republic of the Congo'),
    ('Democratic Republic of Republic of the Congo', 'Democratic Republic of the Congo'),
    ('Timor', 'Timor-Leste'),
    ('Svalbard and Jan Mayen', 'Svalbard'),
    ('Czechia', 'Czech Republic'),
    ('Gambia', 'The Gambia'),
    ('Eswatini', 'Swaziland'),
    ('Micronesia (country)', 'Federated States of Micronesia'),
    ('North Macedonia', 'Macedonia')
]

for pair in replace_countries:
    merged.country = merged.country.replace(pair[0], pair[1])

In [6]:
# Create list of relevant columns to add to geojson

all_sectors = ['Agriculture', 'Land-use change and forestry', 'Waste', 'Industry',
               'Manufacturing and construction', 'Transport', 'Electricity and heat',
               'Buildings', 'Fugitive emissions', 'Other fuel combustion', 'Aviation and shipping']
labels = ['total_ghg', 'co2']+all_sectors

# Create output geojson by adding to the previous one the ghg emissions for each country

for feature in geo['features']:
    try: 
        
        # Map name
        
        country_name = feature['properties']['name']
        country = merged[merged.country == country_name]
        
        # Find and add relevant emissions

        for i, l in enumerate(labels):
            try: 
                feature['properties'][l] = list(country.loc[:, l])
            except IndexError:
                print("INDEX ERROR: ", i, l, country_name)
                
    except KeyError:
        print("Country not found: ", country_name)
        
# Save new geojson

with open("output_geojson.js", "w") as js_file:
    json.dump(geo, js_file)

In [7]:
# Verify geojson

with open("output_geojson.js") as f:
     output = json.loads(f.read())
df = pd.DataFrame(output['features'])
df.drop(columns=['type', 'geometry'], inplace=True)
df = pd.concat([df.drop('properties', axis=1), pd.DataFrame(df['properties'].tolist())], axis=1)
df.head()

,id,name,total_ghg,co2,Agriculture,Land-use change and forestry,Waste,Industry,Manufacturing and construction,Transport,Electricity and heat,Buildings,Fugitive emissions,Other fuel combustion,Aviation and shipping
0,AF,Afghanistan,"[9.58, 9.81, 9.03, 9.11, 9.15, 9.58, 10.61, 11...","[2.024, 1.914, 1.482, 1.487, 1.454, 1.417, 1.3...","[8069999.694824219, 8399999.618530273, 8409999...","[-2390000.104904175, -2390000.104904175, -2390...","[1230000.0190734863, 1320000.0524520874, 13999...","[50000.00074505806, 59999.99865889549, 59999.9...","[409999.9964237213, 400000.0059604645, 370000....","[1669999.9570846558, 1549999.9523162842, 76999...","[170000.00178813934, 170000.00178813934, 15999...","[79999.99821186066, 70000.00029802322, 29999.9...","[280000.0011920929, 239999.994635582, 200000.0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[19999.999552965164, 19999.999552965164, 19999..."
1,AL,Albania,"[11.3, 8.96, 6.95, 6.92, 7.74, 7.58, 7.29, 6.5...","[5.521, 4.29, 2.517, 2.338, 1.924, 2.089, 2.01...","[3549999.952316284, 3240000.009536743, 3170000...","[200000.00298023224, 200000.00298023224, 20000...","[899999.9761581421, 910000.0262260436, 910000....","[439999.9976158142, 360000.01430511475, 170000...","[2150000.095367432, 1419999.9570846558, 759999...","[720000.0286102295, 550000.011920929, 419999.9...","[730000.0190734863, 540000.0214576721, 400000....","[439999.9976158142, 500000.0, 419999.986886978...","[330000.01311302185, 270000.01072883606, 20000...","[1820000.0524520876, 980000.0190734864, 289999...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,DZ,Algeria,"[121.4, 125.57, 126.53, 135.01, 136.96, 143.9,...","[76.738, 78.807, 79.883, 81.944, 86.147, 94.98...","[7869999.885559082, 7619999.885559082, 7940000...","[660000.0262260437, 660000.0262260437, 660000....","[4880000.114440918, 5050000.190734863, 5210000...","[4050000.1907348633, 3859999.895095825, 373000...","[5690000.057220459, 6239999.771118164, 6329999...","[16190000.534057615, 17280000.686645508, 17200...","[23129999.1607666, 24110000.61035156, 24090000...","[6610000.133514404, 7559999.942779541, 7800000...","[52340000.15258789, 53200000.76293945, 5356999...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[2500000.0, 2180000.066757202, 2170000.0762939..."
3,WS,Samoa,"[0.44, 0.45, 0.45, 0.47, 0.49, 0.5, 0.52, 0.52...","[0.088, 0.095, 0.099, 0.103, 0.095, 0.114, 0.1...","[150000.00596046448, 150000.00596046448, 14000...","[109999.99940395357, 109999.99940395357, 10999...","[100000.00149011612, 100000.00149011612, 10000...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[19999.999552965164, 19999.999552965164, 19999...","[50000.00074505806, 50000.00074505806, 59999.9...","[19999.999552965164, 19999.999552965164, 19999...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[19999.999552965164, 19999.999552965164, 19999..."
4,AD,Andorra,"[0.43, 0.43, 0.43, 0.44, 0.44, 0.46, 0.49, 0.5...","[0.407, 0.407, 0.407, 0.41, 0.407, 0.425, 0.45...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[19999.999552965164, 29999.999329447743, 29999...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9999.9997764825...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
